# Analysis of Purchase Lots

In [1]:
import pandas as pd
import datetime

Lot_Data = 'Data_Lots-LYB.csv'
Dividend_Data_Cy = 'Data_Div_Yield_Cy-LYB.csv'
DivPay_Data_Cy = 'Data_Dividends_Pay_Fy-LYB.csv'
today = datetime.date.today()
dividend = 5.36
price = 87
lot_year = 2022
div_paid = [0, 3.93, 5.18, 5.18, 6.43, 6.43 ] # Enter Total Dividends


In [2]:
fwd_div_df = pd.read_csv(Dividend_Data_Cy, index_col='Date')
lot_df = pd.read_csv(Lot_Data, index_col='Date')
pay_df = pd.read_csv(DivPay_Data_Cy, index_col='ExDate')
pay_df.index = pd.to_datetime(pay_df.index)
pay_df = pay_df[pay_df.index.year > lot_year]

fwd_div_df.index = pd.to_datetime(fwd_div_df.index)
lot_df.index = pd.to_datetime(lot_df.index)

pay_df



,Dividend,RecordDate,PayDate,DividendType,Currency
ExDate,,,,,
2024-08-26,1.34,2024-08-26,2024-09-03,Cash Div.,USD
2024-06-03,1.34,2024-06-03,2024-06-10,Cash Div.,USD
2024-03-01,1.25,2024-03-04,2024-03-11,Cash Div.,USD
2023-11-24,1.25,2023-11-27,2023-12-04,Cash Div.,USD
2023-08-25,1.25,2023-08-28,2023-09-05,Cash Div.,USD
2023-05-26,1.25,2023-05-30,2023-06-06,Cash Div.,USD
2023-03-03,1.19,2023-03-06,2023-03-13,Cash Div.,USD


In [3]:
join_df = lot_df.join(fwd_div_df)

In [4]:
record_df = join_df.drop(['SharePrice','Dividend'], axis=1)
record_df['CurrentDividend'] = dividend
record_df['BuyYear'] = record_df.index.year
record_df['CurrentYear'] = today.year
record_df['CurrentPrice'] = price

record_df

,Cost/Share,FwdDiv,FwdDivYield,CurrentDividend,BuyYear,CurrentYear,CurrentPrice
Date,,,,,,,
2024-10-31,87.06,NaN,NaN,5.36,2024,2024,87
2024-02-01,94.30,5.0,0.052849,5.36,2024,2024,87
2023-10-04,93.61,5.0,0.053476,5.36,2023,2024,87
2023-09-21,95.82,5.0,0.052362,5.36,2023,2024,87
2023-08-17,97.74,5.0,0.051578,5.36,2023,2024,87
2023-07-18,90.89,5.0,0.055072,5.36,2023,2024,87


In [5]:
record_df['TotDiv'] = div_paid
record_df['EndValue'] = record_df['CurrentPrice'] + record_df['TotDiv']
record_df['End/Begin'] = record_df['EndValue'] / record_df['Cost/Share']
record_df['Period'] = 1 / (record_df['CurrentYear'] - record_df['BuyYear'])
record_df['CAGR'] = round(((record_df['End/Begin'] ** record_df['Period']) - 1) * 100, 2)

record_df

,Cost/Share,FwdDiv,FwdDivYield,CurrentDividend,BuyYear,CurrentYear,CurrentPrice,TotDiv,EndValue,End/Begin,Period,CAGR
Date,,,,,,,,,,,,
2024-10-31,87.06,NaN,NaN,5.36,2024,2024,87,0.00,87.00,0.999311,inf,-100.00
2024-02-01,94.30,5.0,0.052849,5.36,2024,2024,87,3.93,90.93,0.964263,inf,-100.00
2023-10-04,93.61,5.0,0.053476,5.36,2023,2024,87,5.18,92.18,0.984724,1.0,-1.53
2023-09-21,95.82,5.0,0.052362,5.36,2023,2024,87,5.18,92.18,0.962012,1.0,-3.80
2023-08-17,97.74,5.0,0.051578,5.36,2023,2024,87,6.43,93.43,0.955903,1.0,-4.41
2023-07-18,90.89,5.0,0.055072,5.36,2023,2024,87,6.43,93.43,1.027946,1.0,2.79


In [6]:
yoc_df = record_df[['Cost/Share', 'CurrentDividend']]
yoc_df['YOC'] = round(yoc_df['CurrentDividend'] / yoc_df['Cost/Share'] * 100, 2)

yoc_df

/var/folders/j5/3jkv197574n0l3hvczkvmbw00000gn/T/ipykernel_36296/1063397913.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yoc_df['YOC'] = round(yoc_df['CurrentDividend'] / yoc_df['Cost/Share'] * 100, 2)


,Cost/Share,CurrentDividend,YOC
Date,,,
2024-10-31,87.06,5.36,6.16
2024-02-01,94.30,5.36,5.68
2023-10-04,93.61,5.36,5.73
2023-09-21,95.82,5.36,5.59
2023-08-17,97.74,5.36,5.48
2023-07-18,90.89,5.36,5.90
